In [1]:
#!apt-get install -y libgl1-mesa-dev
!pip3 install --upgrade pip
!pip3 install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio==0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

    100% |################################| 1.7MB 1.2MB/s eta 0:00:01
  Found existing installation: pip 9.0.1
    Not uninstalling pip at /usr/lib/python3/dist-packages, outside environment /usr
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |################################| 1982.1 MB 5.7 kB/s             
     |################################| 17.6 MB 14.1 MB/s            
     |################################| 1.9 MB 15.4 MB/s            


In [2]:
!pip3 install numpy
!pip3 install pandas
!pip3 install multiprocessing
!pip3 install botocore
!pip3 install boto3
!pip3 install scikit-build
!pip3 install cmake
!pip3 install tqdm
!pip3 install opencv-python

     |################################| 108 kB 16.0 MB/s            
  Preparing metadata (setup.py) ... error
  ERROR: Command errored out with exit status 1:
   command: /usr/bin/python3 -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-6z7sv3tm/multiprocessing_6c4cba90d2b4455f8d3b9735374e3bff/setup.py'"'"'; __file__='"'"'/tmp/pip-install-6z7sv3tm/multiprocessing_6c4cba90d2b4455f8d3b9735374e3bff/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base /tmp/pip-pip-egg-info-i_398gaq
       cwd: /tmp/pip-install-6z7sv3tm/multiprocessing_6c4cba90d2b4455f8d3b9735374e3bff/
  Complete output (6 lines):
  Traceback (most recent call last):
    File "<string>", line 1, in <module>
    File "/tmp/pip-install-6z7sv3tm/multiprocess

     |################################| 8.0 MB 10.9 MB/s            
     |################################| 138 kB 17.6 MB/s            
     |################################| 131 kB 10.5 MB/s            
     |################################| 79 kB 17.7 MB/s            
     |################################| 73 kB 4.9 MB/s             
     |################################| 21.5 MB 12.8 MB/s            
     |################################| 60.3 MB 18.4 MB/s            


In [3]:
#確認
import os
os.environ.pop('CUDA_LAUNCH_BLOCKING', None)
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name())
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

True
GeForce RTX 3070


In [4]:
import cv2
import logging
import pandas as pd
import logging
import os
from multiprocessing import Pool, Manager
import functools
import tqdm

In [5]:
#make model
from vision.ssd.mobilenetv3_ssd_lite import create_mobilenetv3_small_ssd_lite
from vision.ssd.config import mobilenetv1_ssd_config
create_net = lambda num: create_mobilenetv3_small_ssd_lite(num, onnx_compatible=True)
config = mobilenetv1_ssd_config

In [6]:
#prepare tranform
from vision.ssd.data_preprocessing import TrainAugmentation, TestTransform
from vision.ssd.ssd import MatchPrior
train_transform = TrainAugmentation(config.image_size, config.image_mean, config.image_std)
target_transform = MatchPrior(config.priors, config.center_variance, config.size_variance, 0.5)
test_transform = TrainAugmentation(config.image_size, config.image_mean, config.image_std)
logging.info("Prepare training datasets.")

In [7]:
#optunaでカリカリにチューニングする
Param = {
    "batch_size" : 8,
    "base_net_lr" : 0.001, #tune
    "num_epochs" : 200,
    "validation_epochs" : 5,
    "t_max" : 100, #tune
    "lr" : 0.01, #tune
    "freeze_base_net" : False, #ファインチューニングするときはTrue,
    "freeze_net" : False,
    "num_workers" : 2, #並列処理 windows対応してないです…
    "momentum" : 0.9, #tune
    "weight_decay" : 5e-4, #tune
    "t_max" : 100, #tune
    "debug_steps" : 100,
}

In [8]:
dataset_path = "./all_images"
check_point_path = "./models/gaku_cornv4_useopenimg_background"
os.makedirs(check_point_path, exist_ok=True)

In [9]:
from vision.utils.misc import str2bool, Timer, freeze_net_layers, store_labels
from torch.utils.data import DataLoader, ConcatDataset
#TODO データセットのライブラリ作る、transformも -> OpenImagesDatasetをたたき台にする 最終的にsub-{}-annotation-bboxに統合する
from vision.datasets.open_images import OpenImagesDataset

datasets = []
#dawnsampling
dataset = OpenImagesDataset(dataset_path,transform=train_transform, target_transform=target_transform, dataset_type="train", balance_data=False)
label_file = os.path.join(check_point_path, "open-images-model-labels.txt")
if os.path.exists(label_file):
    os.remove(label_file)
store_labels(label_file, dataset.class_names)

logging.info(dataset)
num_classes = len(dataset.class_names)
print("class name : ", dataset.class_names)

datasets.append(dataset)

num_classes = len(dataset.class_names)
print(label_file) #models/open-images-model-labels.txtに対象載ってる
train_dataset = ConcatDataset(datasets)
print(len(train_dataset))
train_loader = DataLoader(train_dataset, Param["batch_size"],num_workers=Param["num_workers"],shuffle=True)

['BACKGROUND', 'trafficcone', 'can']
class name :  ['BACKGROUND', 'BACKGROUND', 'can', 'trafficcone']
./models/gaku_cornv4_useopenimg_background/open-images-model-labels.txt
4863


In [10]:
logging.info("Prepare Validation datasets.")
val_dataset = OpenImagesDataset(dataset_path,transform=test_transform, target_transform=target_transform, dataset_type="test")
logging.info(val_dataset)
logging.info("validation dataset size: {}".format(len(val_dataset)))
val_loader = DataLoader(val_dataset,  Param["batch_size"],num_workers=Param["num_workers"],shuffle=False)

['BACKGROUND', 'trafficcone', 'can']


In [11]:
#if fine tuning, change this cell
import itertools

logging.info("Build network.")
net = create_net(num_classes)
min_loss = -10000.0
last_epoch = -1 #pretrain
#model_path = "./models/gaku_cornv1/mbv3-v3-Epoch-15-Loss-6.931226122763849.pth" #no
model_path = "no"
if model_path != "no":
    net.init_from_pretrained_ssd(model_path)
base_net_lr = Param["base_net_lr"]
extra_layers_lr = Param["lr"]
if Param["freeze_base_net"] == True:
    logging.info("Freeze base net.")
    freeze_net_layers(net.base_net)
    params = itertools.chain(net.source_layer_add_ons.parameters(), net.extras.parameters(),
                            net.regression_headers.parameters(), net.classification_headers.parameters())
    params = [
        {'params': itertools.chain(
            net.source_layer_add_ons.parameters(),
            net.extras.parameters()
        ), 'lr': extra_layers_lr},
        {'params': itertools.chain(
            net.regression_headers.parameters(),
            net.classification_headers.parameters()
        )}
    ]
elif Param["freeze_base_net"] == True:
    freeze_net_layers(net.base_net)
    freeze_net_layers(net.source_layer_add_ons)
    freeze_net_layers(net.extras)
    params = itertools.chain(net.regression_headers.parameters(), net.classification_headers.parameters())
    logging.info("Freeze all the layers except prediction heads.")
else:
    params = [
        {'params': net.base_net.parameters(), 'lr': base_net_lr},
        {'params': itertools.chain(
            net.source_layer_add_ons.parameters(),
            net.extras.parameters()
        ), 'lr': extra_layers_lr},
        {'params': itertools.chain(
            net.regression_headers.parameters(),
            net.classification_headers.parameters()
        )}
    ]

In [12]:
#pre-train使うなら argsは頑張る
#if fine tuning, change this cell
"""
if args.resume:
    logging.info(f"Resume from the model {args.resume}")
    net.load(args.resume)
elif args.base_net:
    logging.info(f"Init from base net {args.base_net}")
    net.init_from_base_net(args.base_net)
elif args.pretrained_ssd:
    logging.info(f"Init from pretrained ssd {args.pretrained_ssd}")
    net.init_from_pretrained_ssd(args.pretrained_ssd)
logging.info(f'Took {timer.end("Load Model"):.2f} seconds to load the model.')
"""

'\nif args.resume:\n    logging.info(f"Resume from the model {args.resume}")\n    net.load(args.resume)\nelif args.base_net:\n    logging.info(f"Init from base net {args.base_net}")\n    net.init_from_base_net(args.base_net)\nelif args.pretrained_ssd:\n    logging.info(f"Init from pretrained ssd {args.pretrained_ssd}")\n    net.init_from_pretrained_ssd(args.pretrained_ssd)\nlogging.info(f\'Took {timer.end("Load Model"):.2f} seconds to load the model.\')\n'

In [13]:
net.to(DEVICE)

SSD(
  (base_net): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): hswish()
    (3): Block(
      (se): SeModule(
        (se): Sequential(
          (0): AdaptiveAvgPool2d(output_size=1)
          (1): Conv2d(16, 4, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (2): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (3): ReLU(inplace=True)
          (4): Conv2d(4, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (5): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (6): hsigmoid()
        )
      )
      (conv1): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (nolinear1): ReLU(inplace=True)
      (conv2): Conv2d(16, 16, kernel_

In [14]:
from vision.nn.multibox_loss import MultiboxLoss
from torch.optim.lr_scheduler import CosineAnnealingLR, MultiStepLR

criterion = MultiboxLoss(config.priors, iou_threshold=0.5, neg_pos_ratio=3,
                             center_variance=0.1, size_variance=0.2, device=DEVICE)
optimizer = torch.optim.SGD(params, lr=Param["lr"], momentum=Param["momentum"],
                            weight_decay=Param["weight_decay"])
scheduler = CosineAnnealingLR(optimizer, Param["t_max"], last_epoch=-1)

In [15]:
"""
for _ in range(last_epoch):
    print("skip")
    scheduler.step()
"""

'\nfor _ in range(last_epoch):\n    print("skip")\n    scheduler.step()\n'

In [16]:
def train(loader, net, criterion, optimizer, device, debug_steps=100, epoch=-1):
    net.train(True)
    running_loss = 0.0
    running_regression_loss = 0.0
    running_classification_loss = 0.0
    for i, data in enumerate(loader):
        images, boxes, labels = data
        images = images.to(device)
        boxes = boxes.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        confidence, locations = net(images)
        #print(boxes.shape, labels.shape, confidence.shape, locations.shape)
        regression_loss, classification_loss = criterion(confidence, locations, labels, boxes)  # TODO CHANGE BOXES
        loss = regression_loss + classification_loss
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        running_regression_loss += regression_loss.item()
        running_classification_loss += classification_loss.item()
        if i and i % debug_steps == 0:
            avg_loss = running_loss / debug_steps
            avg_reg_loss = running_regression_loss / debug_steps
            avg_clf_loss = running_classification_loss / debug_steps
            
            logging.info(
                f"Epoch: {epoch}, Step: {i}, " +
                f"Average Loss: {avg_loss:.4f}, " +
                f"Average Regression Loss {avg_reg_loss:.4f}, " +
                f"Average Classification Loss: {avg_clf_loss:.4f}"
            )
            """
            print("epoch: ", epoch)
            print("Average Loss: ", avg_loss)
            print("Average Regression Loss: ", avg_reg_loss)
            print("Average Classification Loss: ", avg_clf_loss)
            """
            running_loss = 0.0
            running_regression_loss = 0.0
            running_classification_loss = 0.0

In [17]:
def test(loader, net, criterion, device):
    print("start test")
    net.eval()
    running_loss = 0.0
    running_regression_loss = 0.0
    running_classification_loss = 0.0
    num = 0
    for _, data in enumerate(loader):
        images, boxes, labels = data
        images = images.to(device)
        boxes = boxes.to(device)
        labels = labels.to(device)
        num += 1

        with torch.no_grad():
            confidence, locations = net(images)
            #print("validation", boxes.shape, labels.shape, confidence.shape, locations.shape)
            regression_loss, classification_loss = criterion(confidence, locations, labels, boxes)
            loss = regression_loss + classification_loss

        running_loss += loss.item()
        running_regression_loss += regression_loss.item()
        running_classification_loss += classification_loss.item()
    return running_loss / num, running_regression_loss / num, running_classification_loss / num

In [ ]:
for epoch in range(last_epoch + 1, Param["num_epochs"]):
    print("from", last_epoch + 1, " now : ", epoch, ' last : ', Param["num_epochs"])
    scheduler.step()
    train(train_loader, net, criterion, optimizer,
            device=DEVICE, debug_steps=Param["debug_steps"], epoch=epoch)
    
    if epoch % Param["validation_epochs"] == 0 or epoch == Param["num_epochs"] - 1:
        val_loss, val_regression_loss, val_classification_loss = test(val_loader, net, criterion, device=DEVICE)
        logging.info(
            f"Epoch: {epoch}, " +
            f"Validation Loss: {val_loss:.4f}, " +
            f"Validation Regression Loss {val_regression_loss:.4f}, " +
            f"Validation Classification Loss: {val_classification_loss:.4f}"
        )
        model_path = os.path.join(check_point_path, "mbv3-Epoch-" + str(epoch).zfill(3) + "-Loss- " + str(val_loss) + ".pth")
        net.save(model_path)
        logging.info(f"Saved model {model_path}")

from 0  now :  0  last :  200


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
/mnt/carryber/pytorch-ssd/vision/transforms/transforms.py:247: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  mode = random.choice(self.sample_options)
/mnt/carryber/pytorch-ssd/vision/transforms/transforms.py:247: VisibleDepreca

start test


/mnt/carryber/pytorch-ssd/vision/transforms/transforms.py:247: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  mode = random.choice(self.sample_options)
/mnt/carryber/pytorch-ssd/vision/transforms/transforms.py:247: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  mode = random.choice(self.sample_options)


from 0  now :  1  last :  200


...

/mnt/carryber/pytorch-ssd/vision/transforms/transforms.py:247: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  mode = random.choice(self.sample_options)
/mnt/carryber/pytorch-ssd/vision/transforms/transforms.py:247: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  mode = random.choice(self.sample_options)


from 0  now :  54  last :  200


/mnt/carryber/pytorch-ssd/vision/transforms/transforms.py:247: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  mode = random.choice(self.sample_options)
/mnt/carryber/pytorch-ssd/vision/transforms/transforms.py:247: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  mode = random.choice(self.sample_options)


from 0  now :  55  last :  200


/mnt/carryber/pytorch-ssd/vision/transforms/transforms.py:247: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  mode = random.choice(self.sample_options)
/mnt/carryber/pytorch-ssd/vision/transforms/transforms.py:247: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  mode = random.choice(self.sample_options)
